In [28]:
import seaborn as sns
import pandas as pd
import numpy as np

df = df = sns.load_dataset("taxis")

declare:
gasoline price/km - 1,4 (default)
auto utilization cost/1000 km - 215 (default)
gasoline used/100km - 5L
tolls - value

In [164]:

def distance_group_column(df: pd.DataFrame)-> pd.DataFrame:
    "returned new column with distance group names"
    df['distance_group'] = np.where(df['distance'] < 5, 'short', 
                        np.where((df['distance'] >= 5) & (df['distance'] < 15), 'medium', 'long'))
    return df

    
def calculate_gasoline_consumption(df: pd.DataFrame)-> pd.DataFrame:
    "calculate gasoline consuption cost"
    gasoline_consuption = 5
    gasoline_price = 1.4
    df = df.assign(gas_expense = lambda x: gasoline_consuption * x['distance']/100*gasoline_price)
    return df
    
def calculate_auto_utilization(df: pd.DataFrame)->pd.DataFrame:
    "calculate auto utilization cost"
    auto_utilization_cost = 215
    df = df.assign(utilization_cost = lambda x: auto_utilization_cost * x['distance']/1000)
    return df

def group_by_distance(df: pd.DataFrame) -> pd.DataFrame:
    "group by distance and calculate profit"
    df = df.groupby(['distance_group'] ).agg(
            count_summons = ('pickup','count'),
            dist_avg = ('distance','mean'),
            dist_total = ('distance','sum'),
            dist_min = ('distance', 'min'),
            dist_max = ('distance', 'max'),
           fare = ('fare','sum'),
           tips = ('tip', 'sum'),
           tolls = ('tolls', 'sum'),
           gas_expense = ('gas_expense', 'sum'),
           auto_utilization_cost = ('utilization_cost', 'sum'),
           hours_sum = ('work_hours','sum')
        ).reset_index()
    return df  

def calculate_work_hours(df: pd.DataFrame)-> pd.DataFrame:
    
    df = df.assign(date_diff = lambda x: x['dropoff'] - x['pickup'])
    df = df.assign(work_hours = lambda x: x['date_diff'].dt.total_seconds() / 60 / 60)
    
    return df

def calculate_profit_by_hour(df: pd.DataFrame)-> pd.DataFrame:
    df = df.assign(profit_per_hour = lambda x: x['total']/x['hours_sum'])
    
    return df
    
    

def calculate_total_profit(df: pd.DataFrame)->pd.DataFrame:
    "Get total profit"
    df = df.assign(total = lambda x: sum([x['fare'],x['tips']-sum([x['tolls'],x['gas_expense'],x['auto_utilization_cost']])]))
    return df



In [165]:
df = df = sns.load_dataset("taxis")

df = distance_group_column(df)
df = calculate_work_hours(df)
df = calculate_gasoline_consumption(df)
df = calculate_auto_utilization(df)
df = group_by_distance(df)
df = calculate_total_profit(df)
df = calculate_profit_by_hour(df)

# most profitable - long distances